In [1]:
import pandas as pd

# Load the datasets
games = pd.read_csv('games.csv')
players = pd.read_csv('players.csv')
player_play = pd.read_csv('player_play.csv')
plays = pd.read_csv('plays.csv')

# Step 1: Merge the datasets
# Merge player_play with plays to get play details
player_play_merged = player_play.merge(plays, on=['gameId', 'playId'])

# Merge with players to get player details
player_play_merged = player_play_merged.merge(players, on='nflId')

# Merge with games to get game outcomes
combined_data = player_play_merged.merge(games, on='gameId')

In [2]:
# Step 3: Key Metrics and Features
# Calculate total rushing yards and passing yards per player
aggregate_metrics = combined_data.groupby('nflId').agg(
    total_rushing_yards=pd.NamedAgg(column='rushingYards', aggfunc='sum'),
    total_passing_yards=pd.NamedAgg(column='passingYards', aggfunc='sum'),
    total_plays=pd.NamedAgg(column='playId', aggfunc='count')
).reset_index()

# Count the number of plays for each game without conflicting names
play_counts = combined_data.groupby('gameId')['playId'].count().reset_index(name='play_count')

# Merge play counts back into the combined_data (avoid naming conflicts)
combined_data = combined_data.merge(play_counts, on='gameId', how='left', suffixes=('', '_y'))

# Calculate average yards gained per play, handle division by zero
combined_data['average_yards_gained'] = combined_data['yardsGained'] / combined_data['play_count']

# Replace infinity values with 0
combined_data['average_yards_gained'].replace([float('inf'), -float('inf')], 0, inplace=True)

# Create additional features: total scores per game
total_scores = combined_data.groupby('gameId').agg(
    home_final_score=pd.NamedAgg(column='homeFinalScore', aggfunc='first'),
    visitor_final_score=pd.NamedAgg(column='visitorFinalScore', aggfunc='first')
).reset_index()

# Rename the score columns to avoid conflicts
total_scores.rename(columns={
    'home_final_score': 'home_final_score_new',
    'visitor_final_score': 'visitor_final_score_new'
}, inplace=True)

# Merge total scores back into the combined data
combined_data = combined_data.merge(total_scores, on='gameId', how='left')

# Display the final aggregated metrics and the combined dataset
print(aggregate_metrics.head())


   nflId  total_rushing_yards  total_passing_yards  total_plays
0  33084                   12                  352           90
1  34452                    2                  232           64
2  35443                    0                    0           64
3  35449                    0                    0           32
4  35452                    0                    0           28


/var/folders/8y/m7b7vwqn1gq3s5gg_9vtr5b80000gn/T/ipykernel_48114/3275139307.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['average_yards_gained'].replace([float('inf'), -float('inf')], 0, inplace=True)


In [4]:
print(combined_data[['gameId', 'nflId', 'average_yards_gained', 'home_final_score_new', 'visitor_final_score_new']].head())

       gameId  nflId  average_yards_gained  home_final_score_new  \
0  2022090800  35472              0.002372                    10   
1  2022090800  42392              0.002372                    10   
2  2022090800  42489              0.002372                    10   
3  2022090800  44875              0.002372                    10   
4  2022090800  44985              0.002372                    10   

   visitor_final_score_new  
0                       31  
1                       31  
2                       31  
3                       31  
4                       31  


In [5]:
print(data.info())

NameError: name 'data' is not defined